In [1]:
!pip install qutip --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 44.6 MB/s eta 0:00:00


Lindbald Master Equation and Hamiltonian

---

$$ \rho=-i[H_{1},\rho]+\gamma_{1D}[2\sigma_{1}\sigma_{1}^{\dagger}-\sigma_{1}^{\dagger}\sigma_{1}\rho-\rho\sigma_{1}^{\dagger}\sigma_{1}] $$

$$ H_{1}=[\omega_{0}+A_{j}(t)]\sigma_{1}^{\dagger}\sigma_{1}-\frac{i\Omega_{R}}{2}(e^{-i\varepsilon t}\sigma_{1}^{\dagger}-H.c) $$

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

%matplotlib inline

In [3]:
gamma_1D = 1.0   # Decay rate
D = 0.5          # Detuning d = (omega_0 - epsilon)
Omega = 5        # Modulation frequency
Omega_R = 0.05   # Rabi frequency
A = 0.025        # Modulation amplitude

sigma = sigmam()        # lowering operator
sigma_dag = sigmap()    # raising operator
n = sigma_dag * sigma   # number operator

In [4]:
def A_modulation(t, args):
    return args['A'] * np.cos(args['Omega'] * t)

def drive_coeff_plus(t, args):
    phase = np.exp(-1j * args['q'] * args['z1'])
    time_phase = np.exp(-1j * args['epsilon'] * t)
    return -1j * args['Omega_R'] / 2 * phase * time_phase

def drive_coeff_minus(t, args):
    phase = np.exp(1j * args['q'] * args['z1'])
    time_phase = np.exp(1j * args['epsilon'] * t)
    return 1j * args['Omega_R'] / 2 * phase * time_phase

In [10]:
H_const = omega_0 * n

H = [
    H_const,
    [n, A_modulation],
    [sigma_dag, drive_coeff_plus],
    [sigma, drive_coeff_minus]]

args = {
    'A': A,
    'Omega': Omega,
    'epsilon': epsilon,
    'Omega_R': Omega_R,
    'q': q,
    'z1': z1
}

psi0 = basis(2, 0)
c_ops = [np.sqrt(gamma_1D) * sigma]
t_final = 10 / gamma_1D
times = np.linspace(0, t_final, 2000)

result = mesolve(H, psi0, times, c_ops, e_ops=[n, sigma_dag, sigma], args=args)

print("[*] MESolve Complete")
print(result)

[*] MESolve Complete
<Result
  Solver: mesolve
  Solver stats:
    method: 'scipy zvode adams'
    init time: 0.00013637542724609375
    preparation time: 0.0003294944763183594
    run time: 0.4335775375366211
    solver: 'Master Equation Evolution'
    num_collapse: 1
  Time interval: [0.0, 100.0] (2000 steps)
  Number of e_ops: 3
  State not saved.
>
